In [1]:
import numpy as np

movieTitles = open("hw8_movieTitles.txt").read().split() # 50x1 mat of movie titles
movieCount = len(movieTitles)
studentPids = open("hw8_studentPIDs.txt").read().split() # 279x1 mat of student PIDs 
studentCount = len(studentPids)
ratings = [] #279x50 mat of userxmovie ratings

for l in open("hw8_ratings.txt"):
    temp = l.rstrip('\n').split()
    ratings.append(temp)

ratings = np.array(ratings)
probZinit = np.loadtxt("hw8_probZ_init.txt") #1x4 
groupCount = len(probZinit) # the number of groups we are considering
probRgivenZ = np.loadtxt("hw8_probRgivenZ_init.txt") #50x4

In [2]:
from collections import Counter
moviePopularityRating = [Counter(ratings[:,t])['1']/(Counter(ratings[:,t])['1']+Counter(ratings[:,t])['0']) for t in range(movieCount)]
for index in np.argsort(moviePopularityRating)[::-1]:
    print(movieTitles[index])

Inception
The_Dark_Knight_Rises
Interstellar
Shutter_Island
The_Martian
The_Social_Network
Now_You_See_Me
12_Years_a_Slave
The_Theory_of_Everything
Gone_Girl
Harry_Potter_and_the_Deathly_Hallows:_Part_2
Toy_Story_3
Black_Swan
Harry_Potter_and_the_Deathly_Hallows:_Part_1
Wolf_of_Wall_Street
The_Avengers
Midnight_in_Paris
Frozen
The_Girls_with_the_Dragon_Tattoo
Django_Unchained
Room
Ex_Machina
Her
The_Revenant
X-Men:_First_Class
The_Great_Gatsby
Jurassic_World
Star_Wars:_The_Force_Awakens
Les_Miserables
Captain_America:_The_First_Avenger
21_Jump_Street
Avengers:_Age_of_Ultron
The_Help
American_Hustle
The_Perks_of_Being_a_Wallflower
Iron_Man_2
The_Hateful_Eight
Fast_Five
The_Hunger_Games
Pitch_Perfect
Thor
Drive
Mad_Max:_Fury_Road
Man_of_Steel
World_War_Z
Bridemaids
Prometheus
Magic_Mike
The_Last_Airbender
Fifty_Shades_of_Grey


In [3]:
probZgivenR = np.zeros(shape=(groupCount, studentCount))

def Estep():
    global probRgivenZ, probZgivenR, probZinit
    for s in range(studentCount):
        sRatings = ratings[s]
        ratedIndices = [i for i, x in enumerate(sRatings) if x == '1' or x == '0']
        probZgivenRt = np.ones(shape=(groupCount))
        denom = 0.0;
        for g in range(groupCount):
            probZgivenRt[g] = probZinit[g];
            for index in ratedIndices:
                if(sRatings[index]=='1'):
                    probZgivenRt[g] *= probRgivenZ[index][g]
                else:
                    probZgivenRt[g] *= (1-probRgivenZ[index][g])
            denom += probZgivenRt[g]
        for g in range(groupCount):
            probZgivenR[g][s] = probZgivenRt[g]/denom

In [4]:
def Mstep():
    global probRgivenZ, probZgivenR, probZinit
    #updating Zinit
    for g in range(groupCount):
        updatedVal = 0.0
        for s in range(studentCount):
            updatedVal += probZgivenR[g][s]
        updatedVal = updatedVal/studentCount
        probZinit[g] = updatedVal
    #updating probRgivenZ
    for g in range(groupCount):
        for m in range(movieCount):
            updatedVal = 0.0
            denom = 0.0
            for s in range(studentCount):
                if(ratings[s][m]=='1'):
                    updatedVal += probZgivenR[g][s]
                elif(ratings[s][m]=='?'):
                    updatedVal += (probZgivenR[g][s]*probRgivenZ[m][g])
                denom += probZgivenR[g][s]
            probRgivenZ[m][g] = updatedVal/denom;

In [5]:
import math
def logLikelihood():
    global probRgivenZ, probZgivenR, probZinit
    denom = studentCount
    ll = 0;
    for s in range(studentCount):
        llisum = 0
        for g in range(groupCount):
            update = probZinit[g]
            ratedIndices = [i for i,x in enumerate(ratings[s]) if x=='1' or x=='0']
            for index in ratedIndices:
                if(ratings[s][index] == '1'):
                    update *= probRgivenZ[index][g]
                else:
                    update *= (1-probRgivenZ[index][g])
            llisum += update
        ll += math.log(llisum)
    ll = ll/denom
    print(ll)

In [6]:
print(logLikelihood()) #At 0th step

-23.68194303920656
None


In [7]:
for i in range(64):
    Estep()
    Mstep()
    print("Iteration: "+str(i+1))
    print(logLikelihood())

Iteration: 1
-14.342138732059837
None
Iteration: 2
-12.909592229693583
None
Iteration: 3
-12.390960695569433
None
Iteration: 4
-12.15061981794007
None
Iteration: 5
-12.031760625835124
None
Iteration: 6
-11.960644019670974
None
Iteration: 7
-11.90884865715703
None
Iteration: 8
-11.867860888836633
None
Iteration: 9
-11.833882674009132
None
Iteration: 10
-11.804593081398394
None
Iteration: 11
-11.7787057143901
None
Iteration: 12
-11.755106316683584
None
Iteration: 13
-11.733413153894418
None
Iteration: 14
-11.713950522388867
None
Iteration: 15
-11.696950121777396
None
Iteration: 16
-11.682204313006867
None
Iteration: 17
-11.668285628739406
None
Iteration: 18
-11.654498444787581
None
Iteration: 19
-11.641607935197339
None
Iteration: 20
-11.630481743512357
None
Iteration: 21
-11.620730278967113
None
Iteration: 22
-11.60984936605054
None
Iteration: 23
-11.59658086131925
None
Iteration: 24
-11.586600888666458
None
Iteration: 25
-11.58018210070515
None
Iteration: 26
-11.576016178631011
None
It

In [17]:
myIndex = studentPids.index('A53244405') #204 index
unseenIndexes = [i for i, x in enumerate(ratings[myIndex]) if x=='?']
probMyUnseen = np.ones(movieCount)
for u in unseenIndexes:
    print(movieTitles[u])
    unseenProb = 0;
    for g in range(groupCount):
        probTemp = probRgivenZ[u][g]
        probTemp *= probZgivenR[g][myIndex]
        unseenProb += probTemp
    probMyUnseen[u] = unseenProb
    print(unseenProb)
indexMovies = np.argsort(probMyUnseen)[::-1]
for i in indexMovies:
    if(i in unseenIndexes):
        print(movieTitles[i])

Black_Swan
0.777160769578
Shutter_Island
0.916768459654
The_Last_Airbender
0.41936899168
Bridemaids
0.40730924267
The_Girls_with_the_Dragon_Tattoo
0.614695520663
Drive
0.415460260347
Midnight_in_Paris
0.813838407899
Prometheus
0.481687423616
Django_Unchained
0.684712426299
Les_Miserables
0.565015875928
21_Jump_Street
0.651450715988
Magic_Mike
0.46759972124
Her
0.561161578191
12_Years_a_Slave
0.751319306016
World_War_Z
0.54134111622
American_Hustle
0.707790157046
Ex_Machina
0.689212495903
Room
0.560915743242
The_Martian
0.832948696225
The_Hateful_Eight
0.404439382717
The_Revenant
0.695193108419
Shutter_Island
The_Martian
Midnight_in_Paris
Black_Swan
12_Years_a_Slave
American_Hustle
The_Revenant
Ex_Machina
Django_Unchained
21_Jump_Street
The_Girls_with_the_Dragon_Tattoo
Les_Miserables
Her
Room
World_War_Z
Prometheus
Magic_Mike
The_Last_Airbender
Drive
Bridemaids
The_Hateful_Eight
